# Verification of the calculations in Section 7
This notebook will verify the calculations in Section 7 of the paper "Incorporating history and deviations in forward-backward splitting" by Hamed Sadeghi, Sebastian Banert, and Pontus Giselsson. To carry out the symbolic calculations, we will make use of the SymPy package.

In [1]:
from sympy import *

First, we will define the parameters of Algorithm 1.

In [2]:
# Cocoercivity parameter of the operator $C$, see Assumption 1.
beta = symbols("beta")

# Input parameters from Algorithm 1; "np1" denotes index $n+1$, "nm1" denotes index $n-1$.
gamma_np1, gamma_n, gamma_nm1, lambda_np1, lambda_n, zeta_n, mu_n, mu_np1, beta_bar = \
    symbols("gamma_{n+1} gamma_n gamma_{n-1} lambda_{n+1} lambda_n zeta_n mu_n mu_{n+1} \\bar{\\beta}")

# Dependent variables from step 2 of Algorithm 1.
alpha_n = mu_n/(lambda_n + mu_n)
alpha_np1 = mu_np1/(lambda_np1 + mu_np1)
alpha_n_bar = gamma_n * mu_n / (gamma_nm1 * (lambda_n + mu_n))
alpha_np1_bar = gamma_np1 * mu_np1 / (gamma_n * (lambda_np1 + mu_np1))
theta_n = (4 - gamma_n * beta_bar) * (lambda_n + mu_n) - 2 * lambda_n ** 2
theta_n_hat = 2 * lambda_n + 2 * mu_n - gamma_n * beta_bar * lambda_n ** 2
theta_n_bar = lambda_n + mu_n - lambda_n ** 2
theta_n_tilde = (lambda_n + mu_n) * gamma_n * beta_bar

# Dependent variable from eq. (38).
theta_n_prime = (2 - gamma_n * beta_bar) * mu_n + 2 * alpha_n_bar * theta_n_bar

Elements of vector spaces are represented by vectors. First we define a set of eight base vectors, and all other vectors are linear combinations of those base vectors.

In [3]:
# Base vectors.
I = eye(8)
y_nm1 = I[0, :]
z_nm1 = I[1, :]
p_nm1 = I[2, :]
x_n   = I[3, :]
u_n   = I[4, :]
v_n   = I[5, :]
p_n   = I[6, :]
x_star = I[7, :]

# Steps 5, 6, and 8 of Algorithm 1 define $y_n$, $z_n$, and $x_{n+1}$ as linear combinations of the base vectors.
y_n = x_n + alpha_n * (y_nm1 - x_n) + u_n
z_n = x_n + alpha_n * (p_nm1 - x_n) + alpha_n_bar * (z_nm1 - p_nm1) + theta_n_bar * gamma_n * beta_bar / theta_n_hat * u_n + v_n
x_np1 = x_n + lambda_n * (p_n - z_n) + alpha_n_bar * lambda_n * (z_nm1 - p_nm1)


Inner products and norms (in the metric associated with $M$) are represented by (Grammian) matrices indicating the coefficients for the inner products between each pair of base vectors.

In [4]:
def inner(x, y):
    return (x.transpose() * y + y.transpose() * x)/2

def normsq(x):
    return inner(x, x)

In [5]:
# These terms are defined in eq. (8).
phi_n = inner((z_n - p_n)/gamma_n, p_n - x_star) + beta_bar/4 * normsq(y_n - p_n)
phi_nm1 = inner((z_nm1 - p_nm1)/gamma_nm1, p_nm1 - x_star) + beta_bar/4 * normsq(y_nm1 - p_nm1)

# The following terms are defined in eq. (3). We will not need an explicit form for $\ell_{n-1}$, so we just leave it as a symbol.
ell_n = theta_n/2 * normsq(p_n - x_n + alpha_n * (x_n - p_nm1) + gamma_n * beta_bar * lambda_n ** 2 / theta_n_hat * u_n - 2 * theta_n_bar / theta_n * v_n) \
    + 2 * mu_n * gamma_n * inner((z_n - p_n)/gamma_n - (z_nm1 - p_nm1)/gamma_nm1, p_n - p_nm1) \
    + mu_n * gamma_n * beta_bar / 2 * normsq(p_n - y_n - (p_nm1 - y_nm1))
ell_nm1 = MatrixSymbol("ell_{n-1}", 8, 8)

# The following terms are defined in eq. (7).
V_np1 = normsq(x_np1 - x_star) + 2 * lambda_np1 * gamma_np1 * alpha_np1 * phi_n + ell_n
V_n = normsq(x_n - x_star) + 2 * lambda_n * gamma_n * alpha_n * phi_nm1 + ell_nm1

The following expression represents the difference of the left-hand side and the right-hand side of the assertion in Theorem 1. Later, we will simplify it to zero in order to show that both sides are equal.

In [6]:
Delta_n = V_np1 - V_n + 2 * gamma_n * (lambda_n - alpha_np1_bar * lambda_np1) * phi_n + ell_nm1 - (lambda_n + mu_n) * (theta_n_tilde/theta_n_hat * normsq(u_n) + theta_n_hat/theta_n * normsq(v_n))

## Proposition 6
All identities are verified by simplifying the expression (lhs - rhs) to zero.

In [7]:
simplify(theta_n - (2 - gamma_n * beta_bar)  * theta_n_bar - theta_n_hat)

0

In [8]:
simplify(theta_n - 2 * theta_n_bar - (2 - gamma_n * beta_bar) * (lambda_n + mu_n))

0

In [9]:
simplify(lambda_n ** 2 * theta_n - theta_n_hat * (lambda_n + mu_n) + 2 * theta_n_bar ** 2)

0

## Lemma 2

In [10]:
lemma_2_i = simplify(p_n - (1 - alpha_n) * x_n - alpha_n * p_nm1 + gamma_n * beta_bar * lambda_n ** 2 / theta_n_hat * u_n - 2 * theta_n_bar / theta_n * v_n)

In [11]:
lemma_2_ii = simplify(p_n - 2 * theta_n_bar / theta_n * z_n + theta_n_tilde / theta_n * y_n - 2 * lambda_n / theta_n * x_n - theta_n_prime / theta_n * p_nm1 + 2 * theta_n_bar * alpha_n_bar / theta_n * z_nm1 - theta_n_tilde * alpha_n / theta_n * y_nm1)

In [12]:
lemma_2_iii = simplify(1/lambda_n * (x_np1 - x_n) + theta_n_tilde / theta_n_hat * u_n + (2 - gamma_n * beta_bar) * (lambda_n + mu_n) / theta_n * v_n)

In [13]:
simplify(lemma_2_i - lemma_2_iii)

Matrix([[0, 0, 0, 0, 0, 0, 0, 0]])

In [14]:
simplify(lemma_2_i - lemma_2_ii)

Matrix([[0, 0, 0, 0, 0, 0, 0, 0]])

In [15]:
simplify(lemma_2_ii - lemma_2_iii)

Matrix([[0, 0, 0, 0, 0, 0, 0, 0]])

## Theorem 1

In [16]:
simplify(Delta_n)

Matrix([
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, 0, 0]])